In [1]:
import json
import torch
from gpt2 import GPT2Model, get_gpt2_cfg
from samplers import Sampler, GreedySampler, TopKSampler, NucleusSampler

device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt_path = './out/ckpt.pt'

In [2]:
def generate(model, idx: torch.Tensor, max_new_tokens: int, context_size: int, sampler: Sampler, eos_id=None) -> torch.Tensor:
    """
    Sampler classlarıyla beraber bir sonraki tokenları oluşturma fonksiyonu.
    """
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        # sadece son token'a odaklanıyoruz çünkü ondan öncekilerin metin üretirken önemi yok.
        # inference yaparken böyle bir mini-optimizasyon yapılabiliyor.
        # indexlemeye anlayabilmek için logitlerin [batch_size, sequence_length, vocab_size] şeklinde olduğunu hatırlamakta fayda var.
        logits = logits[:, -1, :]

        idx_next = sampler.sample(logits)
        
        # batchteki herhangi bir token eos_id'ye eşitse metin üretimini durdur.
        if eos_id and (idx_next == eos_id).any():
            break

        # sample olan indexlerle beraber eski indexleri döndürüyoruz
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1) şeklinde.

    return idx

In [3]:
cfg = get_gpt2_cfg('very_tiny', vocab_size=141)
model = GPT2Model(cfg).to(device)

In [4]:
# eğitlen modelin parametrelerini yüklüyoruz.
ckpt = torch.load(ckpt_path)
model.load_state_dict(ckpt['model'])

<All keys matched successfully>

In [5]:
# tokenizer'ın kullandığı vocab ile encode ve decode fonksiyonlarını oluşturuyoruz.
with open('vocab.json', encoding='utf-8') as json_file:
    vocab = json.load(json_file)

char_to_idx = vocab
idx_to_char = {idx: ch for ch, idx in vocab.items()}

def encode_text(text, char_to_idx):
    return torch.tensor([char_to_idx.get(ch, char_to_idx['<bk>']) for ch in text], dtype=torch.long).unsqueeze(0)

def decode_text(tensor, idx_to_char):
    return ''.join([idx_to_char[idx.item()] for idx in tensor])

In [6]:
start_ctx = "Ey acemi dudaklı yar"
encoded_tensor = encode_text(start_ctx, char_to_idx).to(device)
max_new_tokens = 250
context_size = cfg.block_size

In [ ]:
model.eval()
# şimdi ise eğlenceli olan metin üretme kısmındayız.
# unutmayinki hiperparametrelerle oynayıp daha iyi sonuçlar almak mümkün.
greedy_sampler = GreedySampler()
generated_greedy = generate(model, encoded_tensor, max_new_tokens, context_size, greedy_sampler)

top_k_sampler = TopKSampler(temperature=0.8, k=10)
generated_top_k = generate(model, encoded_tensor, max_new_tokens, context_size, top_k_sampler)

nucleus_sampler = NucleusSampler(temperature=1.0, p=0.9)
generated_nucleus = generate(model, encoded_tensor, max_new_tokens, context_size, nucleus_sampler)

In [8]:
# sonuçları ekrana bastıralım
sampler_name = ['Greedy', 'Top-k', 'Nucleus(Top-p)']
sampler_results = [generated_greedy, generated_top_k, generated_nucleus] 

for name, result in zip(sampler_name, sampler_results):
    decoded_text = decode_text(result[0], idx_to_char) 
    print(f"Sampler ismi: {name}")
    print("------")
    print(decoded_text)
    print() # görsellik için.

Sampler ismi: Greedy
------
Ey acemi dudaklı yarattı
Bir karanlık bir karanlık gibi
Bir karanlık gibi bakmadan geliyor
Bir gün bakmadan bakmadan geliyor
Bir gün bir gün bakmadan geliyor geliyor
Bir gün bir gün bakmadan geliyor geliyor
Bir gün bir gün bakmadan geliyor geliyor
Bir gün bir gün bakmad

Sampler ismi: Top-k
------
Ey acemi dudaklı yardan bilmez mi yar? 

Yalnızlığım bu gemileri de artık düş günlerim.
Göçmeden bende ağlattım.
Yüzümü de sen yüzünde.
Artık sevda kal bıkma kaldım; 
Dalgalandım.
Gölgesi arıyor kalbime...

Denet kalkan olmayı, 
Küllere girmeyi benzemeyi; 
Akar elinden 

Sampler ismi: Nucleus(Top-p)
------
Ey acemi dudaklı yardan arayışı ey Nermin? 
Hak yâdı bir halkas eğniyor bunun bilmiyor...
— Allah kulaklarına size ki geliyor, aldığınız! 
— Ma’bedin çıktığı hep diye müslüman sırlar diye bakıyor...
— Ortak verdik...
— Ne dedin? 
— Sırtından sıkıntınız keder? 
— «Bek»
D



### sonuçlar eh işte... diyelim 🐯